In [1]:
%reload_ext autoreload
%autoreload 2

In [12]:
from fastai.vision import *
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
import numpy as np
import pandas as pd

import torch
from  torch import nn

In [13]:
images_path = '../data/resized_images'
labels_path = '../data/resized_labels.csv'
target_size = 320

In [14]:
y = pd.read_csv(labels_path, index_col='filename')

In [15]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    coord_list = []
    coords = list(y.loc[filename])
    for i in range(len(coords)//2):
        coord_list.append([coords[i*2+1],coords[i*2]])
    return tensor(coord_list)

In [16]:
transforms = get_transforms(
    do_flip=False,
    max_rotate=45, 
    max_zoom=1.5, 
    max_lighting=0.5, 
    max_warp=0.4, 
    p_affine=1., 
    p_lighting=1.
) 

In [17]:
data = (
    PointsItemList
        .from_folder(images_path)
        .split_by_rand_pct(0.15)
        .label_from_func(get_y_func)
        .transform(
            transforms,
            size=320,
            tfm_y=True, 
            remove_out=False, 
            padding_mode='reflection', 
            resize_method=ResizeMethod.PAD
        )
        .databunch()
        .normalize(imagenet_stats)
)

In [18]:
# data.show_batch(3, figsize=(6,6))

In [19]:
learn = cnn_learner(
    data, 
    models.resnet34, 
    loss_func=MSELossFlat()
)#.to_fp16()

In [20]:
learn.path = Path('..')

In [1]:
learn.lr_find()
learn.recorder.plot()

In [16]:
learn.freeze_to(-1)
learn.fit_one_cycle(
    30, 
    slice(1e-2), 
    callbacks=[
        SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-1'),
        EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=3)
    ]
)

epoch,train_loss,valid_loss,time
0,1.042396,0.198158,01:11
1,0.604568,0.094170,01:10
2,0.315669,0.060653,01:11
3,0.202752,0.063023,01:11
4,0.169145,0.056395,01:11
5,0.142665,0.048790,01:11
6,0.105508,0.034817,01:11
7,0.084674,0.025406,01:11
8,0.064814,0.016685,01:11
9,0.073346,0.015950,01:11


Better model found at epoch 0 with valid_loss value: 0.19815798103809357.
Better model found at epoch 1 with valid_loss value: 0.09417025744915009.
Better model found at epoch 2 with valid_loss value: 0.060653455555438995.
Better model found at epoch 4 with valid_loss value: 0.05639549344778061.
Better model found at epoch 5 with valid_loss value: 0.048790331929922104.
Better model found at epoch 6 with valid_loss value: 0.03481731191277504.
Better model found at epoch 7 with valid_loss value: 0.025406161323189735.
Better model found at epoch 8 with valid_loss value: 0.016685357317328453.
Better model found at epoch 9 with valid_loss value: 0.015950292348861694.
Better model found at epoch 10 with valid_loss value: 0.012695029377937317.
Better model found at epoch 11 with valid_loss value: 0.012000830844044685.
Epoch 12: early stopping


In [21]:
learn = learn.load('fastai_model')
learn = learn.to_fp32()
learn.save('fastai_model')

In [22]:
learn.export('models/export.pkl')
learn = load_learner('../models', 'export.pkl')

In [23]:
# learn.show_results(rows=10)

### Sagemaker route

In [25]:
import tarfile
import sagemaker
from sagemaker.utils import name_from_base
from sagemaker.predictor import RealTimePredictor, json_deserializer
from sagemaker.pytorch.model import PyTorchModel

with tarfile.open('../models/model.tar.gz', 'w:gz') as f:
    t = tarfile.TarInfo('models')
    t.type = tarfile.DIRTYPE
    f.addfile(t)
    f.add('../models/export.pkl', arcname='model.pkl')

In [ ]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = f'sagemaker/{name_from_base("facial-features-model")}'
model_artefact = sagemaker_session.upload_data(
    path=str('../models/model.tar.gz'), 
    bucket=bucket, 
    key_prefix=prefix
)

In [ ]:
class ImagePredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(
            endpoint_name, 
            sagemaker_session=sagemaker_session, 
            serializer=None, 
            deserializer=json_deserializer, 
            content_type='image/jpeg'
        )

In [35]:
role = ###
# sagemaker.get_execution_role()

model = PyTorchModel(
    model_data=model_artefact, 
    name=name_from_base("facial-features-model"),
    role=role,
    framework_version='1.2.0', 
    entry_point='serve.py', 
    predictor_cls=ImagePredictor
)

In [36]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-------------------------------------------------------------------------------------------------------------------------------------!

In [11]:
url = "https://mtpcontent-dev.s3.amazonaws.com/ce99060e047d44279e0887270af5afe6.png"
img_bytes = requests.get(url).content
predictor.predict(img_bytes); response

### Bentoml Route

In [26]:
%%writefile pet_regression.py
from bentoml import BentoService, api, env, artifacts 
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

@env(pip_dependencies=['fastai'])
@artifacts([FastaiModelArtifact('pet_regressor')])
class PetRegression(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_regressor.predict(image)
        return str(result)

Overwriting pet_regression.py


In [27]:
# 1) import the custom BentoService defined above
from pet_regression import PetRegression

# 2) `pack` it with required artifacts
service = PetRegression.pack(pet_regressor=learn)

# 3) save your BentoSerivce
saved_path = service.save()

[2019-12-02 23:33:08,301] INFO - BentoService bundle 'PetRegression:20191202233106_823B69' created at: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/bentoml-temp-r3enypu5
[2019-12-02 23:33:14,883] INFO - BentoService bundle 'PetRegression:20191202233106_823B69' created at: /Users/HenryDashwood/bentoml/repository/PetRegression/20191202233106_823B69


In [28]:
from bentoml import load

service = load(saved_path)

print(service.predict(data.get(0)))

[2019-12-02 23:33:15,686] WARNING - Module `pet_regression` already loaded, using existing imported module.
(ImagePoints (320, 320), tensor([[-0.2351, -0.2560],
        [-0.4411, -0.2174],
        [-0.4887,  0.1409],
        [-0.3563,  0.3775],
        [-0.0818,  0.4484],
        [ 0.1933, -0.0060],
        [-0.2732, -0.0603],
        [-0.2323,  0.2372],
        [ 0.1059, -0.0733],
        [ 0.0767,  0.0006],
        [ 0.1394,  0.0717]]), tensor([-0.2351, -0.2560, -0.4411, -0.2174, -0.4887,  0.1409, -0.3563,  0.3775,
        -0.0818,  0.4484,  0.1933, -0.0060, -0.2732, -0.0603, -0.2323,  0.2372,
         0.1059, -0.0733,  0.0767,  0.0006,  0.1394,  0.0717]))


In [29]:
!pip install --upgrade {saved_path}

Processing /Users/HenryDashwood/bentoml/repository/PetRegression/20191202233106_823B69
  Using cached https://files.pythonhosted.org/packages/6d/9e/a612288dd4296b7c40f4e2d6826c94986947afa6918f9088f0de87d71286/BentoML-0.5.2-py3-none-any.whl
  Created wheel for PetRegression: filename=PetRegression-20191202233106_823B69-cp37-none-any.whl size=51628405 sha256=ab2bb7c0e1c72590ba274083fe8d88237d8765ef98cb1198734a499a512e8320
  Stored in directory: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/pip-ephem-wheel-cache-hhyy2m20/wheels/14/a7/4b/c127bb520b70d7e434d0938b5641bd90bbfd85688819d24df1
Successfully built PetRegression
  Found existing installation: BentoML 0.5.3
    Uninstalling BentoML-0.5.3:
      Successfully uninstalled BentoML-0.5.3
  Found existing installation: PetRegression 20191127081517-CDD662
    Uninstalling PetRegression-20191127081517-CDD662:
      Successfully uninstalled PetRegression-20191127081517-CDD662


In [30]:
!PetRegression predict --input=../data/resized_images/Abyssinian_1.jpg

(ImagePoints (320, 320), tensor([[-0.4128,  0.1322],
        [-0.7365,  0.1464],
        [-0.6189,  0.2665],
        [-0.7248,  0.4663],
        [-0.4005,  0.3247],
        [-0.2695,  0.2497],
        [-0.3914,  0.2126],
        [-0.4023,  0.3228],
        [-0.2775,  0.2574],
        [-0.2962,  0.2582],
        [-0.2722,  0.2691]]), tensor([-0.4128,  0.1322, -0.7365,  0.1464, -0.6189,  0.2665, -0.7248,  0.4663,
        -0.4005,  0.3247, -0.2695,  0.2497, -0.3914,  0.2126, -0.4023,  0.3228,
        -0.2775,  0.2574, -0.2962,  0.2582, -0.2722,  0.2691]))


In [31]:
bento_tag = '{name}:{version}'.format(name=service.name, version=service.version)
print(bento_tag)

PetRegression:20191202233106_823B69


In [32]:
!bentoml --verbose deployments delete facial-features-detector --force

[2019-12-02 23:35:46,859] DEBUG - Using BentoML with local Yatai server
[2019-12-02 23:35:49,437] DEBUG - Upgrading tables to the latest revision
Failed to get deployment facial-features-detector for deletion. NOT_FOUND:Deployment "facial-features-detector" in namespace "default" not found


In [33]:
!bentoml --verbose deployment create facial-features-detector --bento {bento_tag} --platform aws-lambda 

[2019-12-02 23:35:51,979] DEBUG - Using BentoML with local Yatai server
[2019-12-02 23:35:52,142] DEBUG - Upgrading tables to the latest revision
[2019-12-02 23:35:56,015] DEBUG - Creating s3 bucket: btml-default-facial-features-detector-32d2cd
[2019-12-02 23:35:57,197] DEBUG - Uploading artifacts to S3 bucket
[2019-12-02 23:35:57,197] DEBUG - This lambda deployment requires uploading artifacts to s3
[2019-12-02 23:35:57,205] DEBUG - Uploading __init__.py to s3 btml-default-facial-features-detector-32d2cd/default/facial-features-detector/artifacts/PetRegression/20191202233106_823B69/__init__.py
[2019-12-02 23:35:57,884] DEBUG - Uploading pet_regressor.pkl to s3 btml-default-facial-features-detector-32d2cd/default/facial-features-detector/artifacts/PetRegression/20191202233106_823B69/pet_regressor.pkl
[2019-12-02 23:36:41,909] DEBUG - Created temporary directory: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/bentoml-temp-ul5ervvj
[2019-12-02 23:36:41,909] DEBUG - Generating c